In [6]:
# when I cropped the landsat tifs in R it reset the nodata values
# i need the nodata value to be np.nan
import os
from glob import glob
import numpy as np
import rasterio 
from rasterio.merge import merge
from rasterio.plot import show
from rasterio.mask import mask
from osgeo import gdal
import sys

import multiprocess
import itertools

In [19]:
# set the nodata value to -9999 
# but actually change all no data pixels to np.nan 
# also need to change all pixels == 0 to np.nan 
home = "/Volumes/GoogleDrive/My Drive/Chapter2_mechanisms_forest_water_cycling"
ndvi_files = glob(os.path.join(home, "Data", "NDVI","Landsat", "Landsat_images_cropped","*.tif"))

for file in ndvi_files: 
    with rasterio.open(file, GDAL_DISABLE_READDIR_ON_OPEN=False) as src: 
        img = np.ma.masked_values(src.read(masked=True), np.nan)
        img[img == 0] = np.nan

        out_name = os.path.join(home, "Data", "NDVI","Landsat", "Landsat_images",os.path.basename(file))
        out_meta = src.meta
        out_meta.update({"driver": "GTiff",
                 "nodata": -9999})
        with rasterio.open(out_name, "w", **out_meta) as dest:
            dest.write(img)


In [7]:
home = "/Volumes/GoogleDrive/My Drive/Chapter2_mechanisms_forest_water_cycling"
ndvi_files = glob(os.path.join(home, "Data", "NDVI","Landsat", "Landsat_images_cropped","*.tif"))

def set_nodata(file): 
    with rasterio.open(file, GDAL_DISABLE_READDIR_ON_OPEN=False) as src: 
        img = np.ma.masked_values(src.read(masked=True), np.nan)
        img[img == 0] = np.nan

        out_name = os.path.join(home, "Data", "NDVI","Landsat", "Landsat_images",os.path.basename(file))
        out_meta = src.meta
        out_meta.update({"driver": "GTiff",
                "nodata": -9999})
        with rasterio.open(out_name, "w", **out_meta) as dest:
            dest.write(img)

if __name__ == "__main__":
   
    # Create a process pool using all cores
    cores = multiprocess.cpu_count() - 3
    p = multiprocess.Pool(cores)

    # parallel map
    stats_lists = p.map(set_nodata, ndvi_files)

